In [61]:
# Для считывания PDF
import PyPDF2
# Для анализа структуры PDF и извлечения текста
from pdfminer.high_level import extract_pages, extract_text
from pdfminer.layout import LTTextContainer, LTChar, LTRect, LTFigure
# Для извлечения текста из таблиц в PDF
import pdfplumber
# Для извлечения изображений из PDF
from PIL import Image
from pdf2image import convert_from_path
# Для выполнения OCR, чтобы извлекать тексты из изображений 
import pytesseract 
# Для удаления дополнительно созданных файлов
import os
from tqdm import tqdm
import json

from nltk.tokenize import word_tokenize

from embedding import E5LargeEmbeddingFunction

import clickhouse_connect
import pandas as pd
import logging
import csv
import re

***

In [2]:
def text_extraction(element):
    # Извлекаем текст из вложенного текстового элемента
    line_text = element.get_text()
    
    # Находим форматы текста
    # Инициализируем список со всеми форматами, встречающимися в строке текста
    line_formats = []
    for text_line in element:
        if isinstance(text_line, LTTextContainer):
            # Итеративно обходим каждый символ в строке текста
            for character in text_line:
                if isinstance(character, LTChar):
                    # Добавляем к символу название шрифта
                    line_formats.append(character.fontname)
                    # Добавляем к символу размер шрифта
                    line_formats.append(character.size)
    # Находим уникальные размеры и названия шрифтов в строке
    format_per_line = list(set(line_formats))
    
    # Возвращаем кортеж с текстом в каждой строке вместе с его форматом
    return (line_text, format_per_line)

# Создаём функцию для вырезания элементов изображений из PDF
def crop_image(element, pageObj):
    # Получаем координаты для вырезания изображения из PDF
    [image_left, image_top, image_right, image_bottom] = [element.x0,element.y0,element.x1,element.y1] 
    # Обрезаем страницу по координатам (left, bottom, right, top)
    pageObj.mediabox.lower_left = (image_left, image_bottom)
    pageObj.mediabox.upper_right = (image_right, image_top)
    # Сохраняем обрезанную страницу в новый PDF
    cropped_pdf_writer = PyPDF2.PdfWriter()
    cropped_pdf_writer.add_page(pageObj)
    # Сохраняем обрезанный PDF в новый файл
    with open('cropped_image.pdf', 'wb') as cropped_pdf_file:
        cropped_pdf_writer.write(cropped_pdf_file)

# Создаём функцию для преобразования PDF в изображения
def convert_to_images(input_file,):
    images = convert_from_path(input_file)
    image = images[0]
    output_file = "PDF_image.png"
    image.save(output_file, "PNG")

# Создаём функцию для считывания текста из изображений
def image_to_text(image_path):
    # Считываем изображение
    img = Image.open(image_path)
    # Извлекаем текст из изображения
    text = pytesseract.image_to_string(img)
    return text

def extract_table(pdf_path, page_num, table_num):
    # Открываем файл pdf
    pdf = pdfplumber.open(pdf_path)
    # Находим исследуемую страницу
    table_page = pdf.pages[page_num]
    # Извлекаем соответствующую таблицу
    table = table_page.extract_tables()[table_num]
    return table

# Преобразуем таблицу в соответствующий формат
def table_converter(table):
    table_string = ''
    # Итеративно обходим каждую строку в таблице
    for row_num in range(len(table)):
        row = table[row_num]
        # Удаляем разрыв строки из текста с переносом
        cleaned_row = [item.replace('\n', ' ') if item is not None and '\n' in item else 'None' if item is None else item for item in row]
        # Преобразуем таблицу в строку
        table_string+=('|'+'|'.join(cleaned_row)+'|'+'\n')
    # Удаляем последний разрыв строки
    table_string = table_string[:-1]
    return table_string

In [3]:
# Находим путь к PDF
pdf_path = 'data/KoAP.pdf'

# создаём объект файла PDF
pdfFileObj = open(pdf_path, 'rb')
# создаём объект считывателя PDF
pdfReaded = PyPDF2.PdfReader(pdfFileObj)

# Создаём словарь для извлечения текста из каждого изображения
text_per_page = {}
# Извлекаем страницы из PDF
for pagenum, page in tqdm(enumerate(extract_pages(pdf_path))):
    
    # Инициализируем переменные, необходимые для извлечения текста со страницы
    pageObj = pdfReaded.pages[pagenum]
    page_text = []
    line_format = []
    text_from_images = []
    text_from_tables = []
    page_content = []
    # Инициализируем количество исследованных таблиц
    table_num = 0
    first_element= True
    table_extraction_flag= False
    # Открываем файл pdf
    pdf = pdfplumber.open(pdf_path)
    # Находим исследуемую страницу
    page_tables = pdf.pages[pagenum]
    # Находим количество таблиц на странице
    tables = page_tables.find_tables()


    # Находим все элементы
    page_elements = [(element.y1, element) for element in page._objs]
    # Сортируем все элементы по порядку нахождения на странице
    page_elements.sort(key=lambda a: a[0], reverse=True)

    # Находим элементы, составляющие страницу
    for i,component in enumerate(page_elements):
        # Извлекаем положение верхнего края элемента в PDF
        pos= component[0]
        # Извлекаем элемент структуры страницы
        element = component[1]
        
        # Проверяем, является ли элемент текстовым
        if isinstance(element, LTTextContainer):
            # Проверяем, находится ли текст в таблице
            if table_extraction_flag == False:
                # Используем функцию извлечения текста и формата для каждого текстового элемента
                (line_text, format_per_line) = text_extraction(element)
                # Добавляем текст каждой строки к тексту страницы
                page_text.append(line_text)
                # Добавляем формат каждой строки, содержащей текст
                line_format.append(format_per_line)
                page_content.append(line_text)
            else:
                # Пропускаем текст, находящийся в таблице
                pass

        # Проверяем элементы на наличие изображений
        if isinstance(element, LTFigure):
            pass
            # # Вырезаем изображение из PDF
            # crop_image(element, pageObj)
            # # Преобразуем обрезанный pdf в изображение
            # convert_to_images('cropped_image.pdf')
            # # Извлекаем текст из изображения
            # image_text = image_to_text('PDF_image.png')
            # text_from_images.append(image_text)
            # page_content.append(image_text)
            # # Добавляем условное обозначение в списки текста и формата
            # page_text.append('image')
            # line_format.append('image')

        # Проверяем элементы на наличие таблиц
        if isinstance(element, LTRect):
            # Если первый прямоугольный элемент
            if first_element == True and (table_num+1) <= len(tables):
                # Находим ограничивающий прямоугольник таблицы
                lower_side = page.bbox[3] - tables[table_num].bbox[3]
                upper_side = element.y1 
                # Извлекаем информацию из таблицы
                table = extract_table(pdf_path, pagenum, table_num)
                # Преобразуем информацию таблицы в формат структурированной строки
                table_string = table_converter(table)
                # Добавляем строку таблицы в список
                text_from_tables.append(table_string)
                page_content.append(table_string)
                # Устанавливаем флаг True, чтобы избежать повторения содержимого
                table_extraction_flag = True
                # Преобразуем в другой элемент
                first_element = False
                # Добавляем условное обозначение в списки текста и формата
                page_text.append('table')
                line_format.append('table')

            # Проверяем, извлекли ли мы уже таблицы из этой страницы
            if element.y0 >= lower_side and element.y1 <= upper_side:
                pass
            elif not isinstance(page_elements[i+1][1], LTRect):
                table_extraction_flag = False
                first_element = True
                table_num+=1


    # Создаём ключ для словаря
    dctkey = 'Page_'+str(pagenum)
    # Добавляем список списков как значение ключа страницы
    text_per_page[dctkey]= [page_text, text_from_tables]
    # text_per_page[dctkey]= [page_text, line_format, text_from_images,text_from_tables, page_content]

# Закрываем объект файла pdf
pdfFileObj.close()

## Удаляем созданные дополнительные файлы
# os.remove('cropped_image.pdf')
# os.remove('PDF_image.png')

# Удаляем содержимое страницы
result = ''.join(text_per_page['Page_0'][4])

630it [14:18,  1.36s/it]


IndexError: list index out of range

In [44]:
new_text_per_page = {}
for i, (key, value) in enumerate(text_per_page.items(), start=1):
    new_text_per_page[i] = value

new_json_path = 'data/new_koap_rf.json'

with open(new_json_path, 'w') as new_json_file:
    json.dump(new_text_per_page, new_json_file)

print("Сохранено в", new_json_path)

Сохранено в data/new_koap_rf.json


In [12]:
new_json_path = 'data/new_koap_rf.json'
with open(new_json_path, 'r') as json_file:
    text_per_page = json.load(json_file)

In [33]:
text_per_page['6']

[['1. Законодательство об административных правонарушениях состоит из настоящего Кодекса \nи  принимаемых  в  соответствии  с  ним  законов  субъектов  Российской  Федерации  об \nадминистративных правонарушениях. \n',
  '2. Настоящий Кодекс основывается на Конституции Российской Федерации, общепризнанных \nпринципах и нормах международного права и международных договорах Российской Федерации. \nЕсли  международным  договором  Российской  Федерации  установлены  иные  правила,  чем \nпредусмотренные  законодательством  об  административных  правонарушениях,  то  применяются \nправила международного договора. \n',
  ' \n',
  'Статья 1.2. Задачи законодательства об административных правонарушениях \n',
  ' \n',
  'Задачами  законодательства  об  административных  правонарушениях  являются  защита \nличности,  охрана  прав  и  свобод  человека  и  гражданина,  охрана  здоровья  граждан,  санитарно-\nэпидемиологического  благополучия  населения,  защита  общественной  нравственности,  охра

In [72]:
def clean_and_join_data(pages):
    cleaned_pages = []
    for page in pages.values():
        cleaned_content = ' '.join(item.replace('\n', '') for sublist in page for item in sublist)
        cleaned_pages.append(cleaned_content)
    return cleaned_pages

# Write the data to a CSV file
with open('data/output.csv', 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    cleaned_data = clean_and_join_data(text_per_page)
    writer.writerow(["Page_Number", "Content"])
    for page_number, page_content in enumerate(cleaned_data, start=1):
        writer.writerow([page_number, page_content])

In [75]:
data = pd.read_csv('data/output.csv')
data.iloc[5]['Content']

'1. Законодательство об административных правонарушениях состоит из настоящего Кодекса и  принимаемых  в  соответствии  с  ним  законов  субъектов  Российской  Федерации  об административных правонарушениях.  2. Настоящий Кодекс основывается на Конституции Российской Федерации, общепризнанных принципах и нормах международного права и международных договорах Российской Федерации. Если  международным  договором  Российской  Федерации  установлены  иные  правила,  чем предусмотренные  законодательством  об  административных  правонарушениях,  то  применяются правила международного договора.    Статья 1.2. Задачи законодательства об административных правонарушениях    Задачами  законодательства  об  административных  правонарушениях  являются  защита личности,  охрана  прав  и  свобод  человека  и  гражданина,  охрана  здоровья  граждан,  санитарно-эпидемиологического  благополучия  населения,  защита  общественной  нравственности,  охрана окружающей государственной  власти, общественного 

***

In [78]:
client = clickhouse_connect.get_client(host='fre1otkh1b.europe-west4.gcp.clickhouse.cloud', port=8443, username='default', password='.SAxpSMBh27IT')

In [79]:
def get_nltk_chunks(text, chunk_size=200, chunk_overlap=100, language="russian"):
    words = word_tokenize(text, language=language)

    chunks = []

    end_index = chunk_size
    while end_index - chunk_size < len(words):
        sentence = ''
        tokens_counter = 0
        for word in words[end_index - chunk_size:end_index]:
            if word in ',.?!:)»' or (sentence and sentence[-1] in '«('):
                sentence += word
            else:
                sentence += " " + word
            tokens_counter += 1

        if tokens_counter < chunk_size and chunks:
            words = word_tokenize(chunks[-1] + sentence)[-chunk_size:]
            sentence = ''
            for word in words:
                if word in ',.?!:)»' or (sentence and sentence[-1] in '«('):
                    sentence += word
                else:
                    sentence += " " + word
                    
        chunks.append(sentence)
        end_index += chunk_size - chunk_overlap

    return chunks

In [80]:
logging.basicConfig(level=logging.INFO, filename='db.log', filemode="a",
                       format="%(asctime)s %(levelname)s %(message)s")

emb_func = E5LargeEmbeddingFunction()

emb_func.change_mode(new_mode='passage')

In [82]:
total_rows = len(data)
with tqdm(total=total_rows, desc='Processing rows...') as pbar:
    for index, row in data.iterrows():
        try:
            if index % 25 == 0:
                logging.info(f'{index} done')
            page = row.Page_Number
            text = row.Content

            chunks = get_nltk_chunks(text, chunk_size=400, chunk_overlap=100)

            insert_data = []
            for i, chunk in enumerate(chunks):
                insert_data.append((index, i, page, *emb_func(chunk)))

            client.insert('index_texts_final', insert_data, column_names=['page_num', 'page_text', 'embedding'])

            pbar.update(1)  # Update tqdm progress bar
        except Exception as e:
            logging.error(index)

Processing rows...:   0%|          | 0/630 [00:00<?, ?it/s]

: 